In [ ]:
import pandas as pd
import networkx as nx
import numpy as np
import json
from collections import defaultdict
import matplotlib.pyplot as plt

# Step 1: Read data and calculate dependency weights
def calculate_dependency_weights(df):
    """
    Calculate dependency weights w_ij = t_ij / Σ_j t_ij
    where t_ij is the commuting flow from i to j
    """
    # Calculate total outflow for each origin city
    total_outflow = df.groupby('ORIGIN_TTWA')['TOTAL_FLOW'].sum()
    
    # Calculate dependency weights
    df['dependency_weight'] = df.apply(
        lambda row: row['TOTAL_FLOW'] / total_outflow[row['ORIGIN_TTWA']], 
        axis=1
    )
    
    return df

# Read data
df = pd.read_csv('ttwa_od_matrix_cross_city_only.csv')
df_with_weights = calculate_dependency_weights(df)

print(f"Data overview: {len(df)} OD records, {df['ORIGIN_TTWA'].nunique()} origin cities")
print(f"Dependency weight range: {df_with_weights['dependency_weight'].min():.6f} - {df_with_weights['dependency_weight'].max():.6f}")

In [ ]:
# Step 2: Build directed commuting network
def build_commuting_network(df_weights):
    """Build directed commuting network"""
    G = nx.DiGraph()
    
    # Add edges with dependency weights
    for _, row in df_weights.iterrows():
        G.add_edge(
            row['ORIGIN_TTWA'], 
            row['DEST_TTWA'], 
            weight=row['dependency_weight'],
            flow=row['TOTAL_FLOW']
        )
    
    return G

G = build_commuting_network(df_with_weights)
print(f"Network scale: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

In [ ]:
# Assume you have already run the previous code and obtained G and df_with_weights

print("--- Starting calculation of network centrality metrics ---")

# --- Step 3: Calculate weighted in-degree (Weighted In-Degree) ---
# G.in_degree() returns a special object containing (node, degree) pairs
# We use weight='flow' to specify weights, which corresponds to the 'flow' attribute you added to edges
# If you want to use 'dependency_weight' as weight, just change weight='flow' to weight='dependency_weight'
print("   Step 1/4: Calculating weighted in-degree...")
in_degree_weighted = dict(G.in_degree(weight='flow'))

# --- Step 4: Convert results to Pandas DataFrame ---
print("   Step 2/4: Organizing results into table format...")
df_metrics = pd.DataFrame.from_dict(
    in_degree_weighted, 
    orient='index', 
    columns=['in_degree_weighted']
)
df_metrics.index.name = 'TTWA_Code'
df_metrics.reset_index(inplace=True)

# --- Step 5: Add TTWA names for easier reading ---
# We extract the correspondence between TTWA codes and names from your original df_with_weights data
# Here we assume the destination columns are named 'DEST_TTWA' and 'DEST_TTWA_NAME', please modify according to your actual situation
print("   Step 3/4: Adding TTWA names...")
try:
    ttwa_names = df_with_weights[['DEST_TTWA', 'DEST_TTWA_NAME']].drop_duplicates().rename(
        columns={'DEST_TTWA': 'TTWA_Code', 'DEST_TTWA_NAME': 'TTWA_Name'}
    )
    # Merge names into metrics table
    final_df = pd.merge(ttwa_names, df_metrics, on='TTWA_Code', how='right')

except KeyError:
    print("   Warning: 'DEST_TTWA_NAME' column not found, output results will not include TTWA names.")
    final_df = df_metrics


# --- Step 6: Save to CSV file ---
OUTPUT_FILE = 'inter_city_in_degree_metrics.csv'
print(f"   Step 4/4: Saving results to '{OUTPUT_FILE}'...")
final_df.to_csv(OUTPUT_FILE, index=False)

print("\nSuccess! City status metrics have been calculated.")
print(f"Output file: {OUTPUT_FILE}")
print("\nFile content preview:")
print(final_df.sort_values('in_degree_weighted', ascending=False).head())

In [ ]:
# run_diagnostics.py

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import pearsonr
import warnings

# Ignore some minor warnings that may appear during plotting
warnings.filterwarnings('ignore')

def analyze_single_model(df, model_vars, model_name):
    """
    Perform correlation matrix visualization and VIF analysis for a single model.
    """
    print(f"\n>>> Analyzing {model_name}...")
    
    # Extract data required for the model and remove rows containing missing values based on these columns
    model_data = df[model_vars].dropna()
    print(f"   Number of valid observations for this model: {len(model_data)}")
    
    # --- Step 1: Correlation matrix ---
    print(f"\n>>> Step 1/2: Correlation analysis for {model_name}...")
    
    corr_matrix = model_data.corr()
    print(f"--- {model_name} Correlation matrix values ---")
    print(corr_matrix.round(4))

    # Find highly correlated independent variable pairs (check only among independent variables)
    independent_vars_only = [var for var in model_vars if var != 'log_in_degree_weighted']
    high_corr_threshold = 0.7
    high_corr_found = False
    for i in range(len(independent_vars_only)):
        for j in range(i + 1, len(independent_vars_only)):
            var1 = independent_vars_only[i]
            var2 = independent_vars_only[j]
            corr_val = abs(corr_matrix.loc[var1, var2])
            if corr_val > high_corr_threshold:
                print(f"Warning: High correlation found among independent variables: {var1} and {var2} (r = {corr_matrix.loc[var1, var2]:.4f})")
                high_corr_found = True
    
    if not high_corr_found:
        print(f"No high correlation found among independent variables (all |r| <= {high_corr_threshold})")

    # --- Step 2: VIF test (for independent variables only) ---
    print(f"\n>>> Step 2/2: Calculating VIF values for {model_name} independent variables...")
    
    vif_data = model_data[independent_vars_only]
    
    if len(independent_vars_only) < 2:
        print(f"   Info: {model_name} has fewer than 2 independent variables, no need to calculate VIF.")
        return
    
    # Calculate VIF
    vif_result = pd.DataFrame()
    vif_result["Variable"] = vif_data.columns
    vif_result["VIF"] = [variance_inflation_factor(vif_data.values, i) for i in range(vif_data.shape[1])]
    
    print(f"\n--- {model_name} VIF test results ---")
    print(vif_result.to_string(index=False))
    
    # Provide diagnostic conclusions based on VIF values
    if (vif_result['VIF'] >= 5).any():
        print(f"Reminder: Variables in {model_name} have VIF values greater than or equal to 5, please interpret regression coefficients carefully.")
    else:
        print("No serious multicollinearity issues, regression analysis can be performed safely.")

def main():
    """
    Main function to read final data and test both models.
    """
    # --- 1. Read final data file generated by previous script ---
    INPUT_DATA_FILE = 'master_analysis_data_final_cleaned.csv'
    
    print(f"\n>>> Reading final dataset '{INPUT_DATA_FILE}'...")
    try:
        df = pd.read_csv(INPUT_DATA_FILE)
        print(f"   Data read successfully, contains {len(df)} final observations.")
    except FileNotFoundError:
        print(f"Error: File '{INPUT_DATA_FILE}' not found. Please run the data cleaning script first.")
        return

    # Create log-transformed dependent variable
    if 'in_degree_weighted' in df.columns:
        df['log_in_degree_weighted'] = np.log1p(df['in_degree_weighted'])
    else:
        print("Error: 'in_degree_weighted' column not found.")
        return

    # --- 2. Define variable sets for both models ---
    # Model 1: Using Gamma (Centripetality)
    model1_vars = [
        'log_in_degree_weighted',
        'Centripetality_Gamma_mean',
        'jobs_density'
    ]
    
    # Model 2: Using Lambda (Anisotropy)
    model2_vars = [
        'log_in_degree_weighted',
        'Anisotropy_Lambda_mean',
        'jobs_density'
    ]
    
    # Check if all variables exist
    all_vars = list(set(model1_vars + model2_vars))
    missing_vars = [var for var in all_vars if var not in df.columns]
    if missing_vars:
        print(f"Error: The following key variables do not exist in the data: {missing_vars}")
        return
    
    print(f"All variables exist, starting to test both models separately...")

    # --- 3. Analyze Model 1 ---
    print("\n" + "="*60)
    print("Model 1 analysis: log(InDegree) ~ Centripetality_Gamma + jobs_density")
    print("="*60)
    analyze_single_model(df, model1_vars, "Model 1 (Gamma)")
    
    # --- 4. Analyze Model 2 ---
    print("\n" + "="*60)
    print("Model 2 analysis: log(InDegree) ~ Anisotropy_Lambda + jobs_density")
    print("="*60)
    analyze_single_model(df, model2_vars, "Model 2 (Lambda)")

    print("\n" + "="*60)
    print("All tests completed.")
    print("="*60)


if __name__ == '__main__':
    main()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')

def create_beautiful_correlation_matrix():
    """
    Create beautiful correlation matrix visualization with different color schemes
    """
    # Read data
    INPUT_DATA_FILE = 'master_analysis_data_final_cleaned.csv'
    
    try:
        df = pd.read_csv(INPUT_DATA_FILE)
        print(f"Data read successfully, contains {len(df)} observations.")
    except FileNotFoundError:
        print(f"Error: File '{INPUT_DATA_FILE}' not found.")
        return

    # Create log-transformed dependent variable
    if 'log_in_degree_weighted' not in df.columns:
        df['log_in_degree_weighted'] = np.log1p(df['in_degree_weighted'])

    # Define analysis variables and labels
    variables = [
        'log_in_degree_weighted',
        'Centripetality_Gamma_mean', 
        'Anisotropy_Lambda_mean',
        'jobs_density'
    ]
    
    # Beautiful variable labels
    variable_labels = {
        'log_in_degree_weighted': 'log(In-Degree)',
        'Centripetality_Gamma_mean': 'Centripetality (Γ)',
        'Anisotropy_Lambda_mean': 'Anisotropy (Λ)',
        'jobs_density': 'Job Density'
    }
    
    # Extract data and remove missing values
    analysis_data = df[variables].dropna()
    
    # Calculate correlation matrix
    corr_matrix = analysis_data.corr()
    
    # Rename row and column labels
    corr_matrix_labeled = corr_matrix.copy()
    corr_matrix_labeled.index = [variable_labels[var] for var in corr_matrix.index]
    corr_matrix_labeled.columns = [variable_labels[var] for var in corr_matrix.columns]
    
    # Set figure style
    plt.style.use('default')
    
    # Create figure
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # ============================================================================
    # Modified color scheme
    # ============================================================================
    sns.heatmap(
        corr_matrix_labeled, 
        annot=True,
        cmap='coolwarm',     # Recommended scheme 1: classic red-white-blue color scheme
        # cmap='RdBu_r',     # Alternative scheme: another classic red-white-blue
        # cmap='PuOr',       # Alternative scheme: soft purple-white-orange
        # cmap='vlag',       # Alternative scheme: modern red-gray-blue
        center=0,
        square=True,
        fmt='.3f',
        cbar_kws={"shrink": .8, "label": "Correlation Coefficient"},
        linewidths=0.5,
        linecolor='white',
        annot_kws={'size': 12, 'weight': 'bold', 'color': 'black'},
        vmin=-1, vmax=1
    )
    
    # Beautification settings
    ax.set_title('Correlation Matrix (General Mobility Network)', fontsize=16, fontweight='bold', pad=20)
    plt.xticks(rotation=45, ha='right', fontsize=11)
    plt.yticks(rotation=0, fontsize=11)
    
    # Adjust layout
    plt.tight_layout()
    
    # Display figure
    plt.show()
    
    # Print numerical matrix
    print("\nCorrelation matrix values:")
    print("=" * 50)
    print(corr_matrix_labeled.round(3))
    
    # Significance test
    print("\nCorrelation significance test:")
    print("=" * 50)
    
    for i, var1 in enumerate(variables):
        for j, var2 in enumerate(variables):
            if i < j:  # Avoid duplication
                corr, p_value = pearsonr(analysis_data[var1], analysis_data[var2])
                
                # Significance markers
                if p_value < 0.001:
                    sig_level = "***"
                elif p_value < 0.01:
                    sig_level = "**"
                elif p_value < 0.05:
                    sig_level = "*"
                else:
                    sig_level = "ns"
                
                print(f"{variable_labels[var1]} ↔ {variable_labels[var2]}:")
                print(f"   r = {corr:.3f}, p = {p_value:.4f} {sig_level}")

# Run function
create_beautiful_correlation_matrix()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

# --- 1. Load final data file ---
try:
    df = pd.read_csv('master_analysis_data_final_cleaned.csv')
    print("--- Final diagnosis: Manual VIF calculation and visualization ---")
except FileNotFoundError:
    print("Error: Cannot find 'master_analysis_data_final_cleaned.csv'")
    exit()

# --- 2. Prepare Model 2 independent variables data ---
model2_vars = ['Anisotropy_Lambda_mean', 'jobs_density']
data = df[model2_vars].dropna()

# --- 3. Manual VIF calculation ---
# The principle of VIF calculation is: use one independent variable as dependent variable, others as independent variables, perform regression
# Then use the R² value from regression to calculate VIF

# Model A: Anisotropy ~ Job Density
y = data['Anisotropy_Lambda_mean']
X = data['jobs_density']
X = sm.add_constant(X) # Regression needs to add constant term

model = sm.OLS(y, X).fit()
rsquared = model.rsquared

# VIF = 1 / (1 - R²)
manual_vif = 1 / (1 - rsquared)

print("\n" + "="*50)
print("Decisive test results:")
print(f"  1. R-squared from regression of 'Anisotropy' on 'Job Density': {rsquared:.6f}")
print(f"  2. Manual VIF calculated from this R-squared: {manual_vif:.6f}")
print("="*50 + "\n")


# --- 4. Visualize relationship between two variables ---
print(">>> Generating relationship plot between Anisotropy_Lambda_mean and jobs_density...")
plt.figure(figsize=(10, 7))

# Use seaborn's regplot, which plots both scatter plot and linear regression line
sns.regplot(
    x='jobs_density', 
    y='Anisotropy_Lambda_mean', 
    data=data,
    scatter_kws={'alpha': 0.6},
    line_kws={'color': 'red', 'linewidth': 2}
)

plt.title('Scatter Plot with Regression Line', fontsize=16)
plt.xlabel('Job Density', fontsize=12)
plt.ylabel('Anisotropy (Lambda)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

# --- 5. Conclusion ---
print("\n--- Diagnostic conclusion ---")
print(f"The correlation coefficient r ≈ 0.06 you saw earlier should theoretically give VIF = 1.004.")
print(f"Our manually calculated VIF is {manual_vif:.4f}.")
if abs(manual_vif - 1.004) < 0.01:
    print("Manual calculation results match theoretical values. This indicates that the statsmodels VIF function may behave abnormally in your environment.")
    print("   You can adopt the manually calculated VIF value as the final basis. Conclusion: Model 2 has no multicollinearity issues.")
else:
    print("Manual calculation results do not match theoretical values but are close to VIF function results.")
    print("   This indicates that there may be extreme outliers or special distributions in the data, seriously affecting the stability of linear regression. Please carefully examine the scatter plot above for clues.")

In [ ]:
# This step can be added to your merge_and_analyze.py or advanced_analysis.py script
# Or as a new preparation script

import geopandas as gpd
# 1. Load your main analysis data
master_df = pd.read_csv('master_analysis_data_final.csv')

# 2. Load TTWA geographic boundary file
TTWA_BOUNDARY_FILE = 'boundary/Travel_to_Work_Areas_Dec_2011_FCB_in_United_Kingdom_2022.geojson'
ttwa_gdf = gpd.read_file(TTWA_BOUNDARY_FILE)

# 3. Calculate centroid coordinates for each TTWA
ttwa_gdf['centroid'] = ttwa_gdf.geometry.centroid
ttwa_gdf['centroid_x'] = ttwa_gdf.centroid.x
ttwa_gdf['centroid_y'] = ttwa_gdf.centroid.y

# 4. Merge coordinates into main analysis data
# We need a common key, here assuming it's TTWA name
master_df_geo = pd.merge(master_df, ttwa_gdf[['TTWA11NM', 'centroid_x', 'centroid_y']], left_on='TTWA_Name', right_on='TTWA11NM', how='left')

# Check merged data
print(master_df_geo.head())

#### New analysis

In [ ]:
# run_regression_analysis.py

import pandas as pd
import statsmodels.formula.api as smf

def run_regression_analysis():
    """
    This script loads the final cleaned data and runs two independent multiple linear regression models,
    examining the relationship between Gamma and Lambda with in-degree respectively, while controlling for jobs_density.
    """
    print("--- Running regression analysis ---")

    # --- 1. File path ---
    # This file should be the output from your previous cleaning script
    INPUT_DATA_FILE = 'master_analysis_data_final_cleaned.csv' 

    # --- 2. Read data ---
    print(f"\n>>> Reading analysis data '{INPUT_DATA_FILE}'...")
    try:
        df = pd.read_csv(INPUT_DATA_FILE)
        print("   Data read successfully.")
        print(f"   Final dataset for analysis contains {len(df)} TTWAs.")
    except FileNotFoundError:
        print(f"Error: File '{INPUT_DATA_FILE}' not found.")
        print("   Please ensure you have successfully run the data cleaning and merging script and generated this file.")
        return

    # --- 3. Run first regression model (using Gamma) ---
    print("\n>>> Running Model 1: using Centripetality_Gamma_mean...")
    
    # Define Model 1 formula (y ~ x1 + x2)
    formula1 = 'in_degree_weighted ~ Centripetality_Gamma_mean + jobs_density'
    
    try:
        # Create and fit OLS model
        model1 = smf.ols(formula=formula1, data=df).fit()
        
        # Print complete regression results summary
        print("\n--- Model 1: Regression analysis results (using Gamma) ---")
        print(model1.summary())
        print("------------------------------------------")
        
    except Exception as e:
        print(f"Error running Model 1: {e}")


    # --- 4. Run second regression model (using Lambda) ---
    print("\n>>> Running Model 2: using Anisotropy_Lambda_mean...")

    # Define Model 2 formula
    formula2 = 'in_degree_weighted ~ Anisotropy_Lambda_mean + jobs_density'
    
    try:
        # Create and fit OLS model
        model2 = smf.ols(formula=formula2, data=df).fit()
        
        # Print complete regression results summary
        print("\n--- Model 2: Regression analysis results (using Lambda) ---")
        print(model2.summary())
        print("------------------------------------------")
    
    except Exception as e:
        print(f"Error running Model 2: {e}")

# --- Run main function ---
run_regression_analysis()

In [ ]:
# run_log_transformed_regression.py

import pandas as pd
import numpy as np  # Import numpy library for mathematical transformations
import statsmodels.formula.api as smf

def run_log_transformed_regression():
    """
    This script loads the final cleaned data, performs log transformation on the dependent variable 'in_degree_weighted',
    then re-runs two multiple linear regression models to obtain more robust results.
    """
    print("--- Running log-transformed regression analysis (more robust models) ---")

    # --- 1. File path ---
    INPUT_DATA_FILE = 'master_analysis_data_final_cleaned.csv'

    # --- 2. Read data ---
    print(f"\n>>> Reading analysis data '{INPUT_DATA_FILE}'...")
    try:
        df = pd.read_csv(INPUT_DATA_FILE)
        print("   Data read successfully.")
    except FileNotFoundError:
        print(f"Error: File '{INPUT_DATA_FILE}' not found.")
        return

    # --- 3. Core improvement: log transform the dependent variable ---
    print("\n>>> Log transforming dependent variable 'in_degree_weighted'...")
    
    # We use np.log1p for transformation, which calculates log(1 + x).
    # This is a professional data processing technique that can robustly handle potential 0 values in original data.
    # Using np.log(0) directly would cause errors.
    log_target_col = 'log_in_degree_weighted'
    original_target_col = 'in_degree_weighted'

    if original_target_col in df.columns:
        df[log_target_col] = np.log1p(df[original_target_col])
        print(f"   Created new column '{log_target_col}' for analysis.")
    else:
        print(f"Error: Original dependent variable column '{original_target_col}' not found.")
        return

    # --- 4. Run first regression model (using Gamma) ---
    print("\n>>> Running Model 1 (using Gamma)...")
    
    # Note: The dependent variable on the left side of the formula has been changed to the new log-transformed column
    formula1 = f'{log_target_col} ~ Centripetality_Gamma_mean + jobs_density'
    
    try:
        model1 = smf.ols(formula=formula1, data=df).fit()
        print("\n--- Model 1: Regression analysis results (log-transformed, using Gamma) ---")
        print(model1.summary())
        print("--------------------------------------------------")
    except Exception as e:
        print(f"Error running Model 1: {e}")

    # --- 5. Run second regression model (using Lambda) ---
    print("\n>>> Running Model 2 (using Lambda)...")

    formula2 = f'{log_target_col} ~ Anisotropy_Lambda_mean + jobs_density'
    
    try:
        model2 = smf.ols(formula=formula2, data=df).fit()
        print("\n--- Model 2: Regression analysis results (log-transformed, using Lambda) ---")
        print(model2.summary())
        print("--------------------------------------------------")
    except Exception as e:
        print(f"Error running Model 2: {e}")

# --- Run main function ---
run_log_transformed_regression()

In [ ]:
# moran_i_test_final.py

import pandas as pd
import geopandas as gpd
import statsmodels.formula.api as smf
import numpy as np
from libpysal.weights import Queen
from esda.moran import Moran

def run_final_spatial_diagnostics():
    """
    This script is the final diagnostic step for comprehensive network analysis.
    Core logic:
    1. Load data and ensure geographic coordinate system is EPSG:2770.
    2. Run the final confirmed log-transformed OLS regression model.
    3. Extract model residuals.
    4. Perform Moran's I spatial autocorrelation test on residuals.
    5. Based on test results, clarify next action recommendations (whether to proceed with MGWR).
    """
    print("--- Final spatial diagnostics for comprehensive network regression model ---")

    # --- 1. File paths and model configuration ---
    ANALYSIS_DATA_FILE = 'master_analysis_data_final_cleaned.csv' 
    TTWA_BOUNDARY_FILE = "boundary/Travel_to_Work_Areas_Dec_2011_FCB_in_United_Kingdom_2022.geojson"
    
    # Final confirmed model formula
    FINAL_MODEL_FORMULA = 'log_in_degree_weighted ~ Anisotropy_Lambda_mean + jobs_density'

    # --- 2. Load, transform coordinate system and merge data ---
    print("\n>>> Loading data and setting correct coordinate system (EPSG:2770)...")
    try:
        df_analysis = pd.read_csv(ANALYSIS_DATA_FILE)
        gdf_ttwa = gpd.read_file(TTWA_BOUNDARY_FILE)
        
        # Core step: Transform geographic data to EPSG:2770
        gdf_ttwa_projected = gdf_ttwa.to_crs('EPSG:2770')
        
        # Merge analysis data to geographic data
        gdf_analysis = gdf_ttwa_projected.merge(df_analysis, left_on='TTWA11NM', right_on='TTWA_Name', how='inner')
        
        print(f"   Data loading, coordinate system transformation and merging successful, {len(gdf_analysis)} TTWAs for spatial analysis.")

    except FileNotFoundError as e:
        print(f"Error: File {e.filename} not found. Please check path.")
        return
    except Exception as e:
        print(f"Error during data loading or merging: {e}")
        return

    # --- 3. Run final OLS regression model and extract residuals ---
    print("\n>>> Running final log-transformed OLS model to obtain residuals...")
    
    # Core step: Log transform dependent variable
    gdf_analysis['log_in_degree_weighted'] = np.log1p(gdf_analysis['in_degree_weighted'])
    
    try:
        model = smf.ols(formula=FINAL_MODEL_FORMULA, data=gdf_analysis).fit()
        gdf_analysis['residuals'] = model.resid
        print("   OLS model run completed, residuals successfully extracted.")
    except Exception as e:
        print(f"Error running OLS model: {e}")
        return

    # --- 4. Build spatial weight matrix (W) ---
    print("\n>>> Building spatial weight matrix (W) based on adjacency relationships...")
    try:
        # Use Queen Contiguity: neighbors if they share even one vertex
        w = Queen.from_dataframe(gdf_analysis)
        w.transform = 'r' # Row standardization
        print("   Spatial weight matrix (W) built successfully.")
    except Exception as e:
        print(f"Error building spatial weight matrix: {e}")
        return

    # --- 5. Calculate Moran's I ---
    print("\n>>> Performing Moran's I test on model residuals...")
    try:
        moran = Moran(gdf_analysis['residuals'], w)
        print("   Moran's I calculation completed.")
    except Exception as e:
        print(f"Error calculating Moran's I: {e}")
        return

    # --- 6. Report results and final recommendations ---
    print("\n--- Spatial autocorrelation test results ---")
    print(f"Moran's I statistic: {moran.I:.4f}")
    print(f"p-value (simulation): {moran.p_sim:.4f}")
    print("----------------------------")

    print("\n--- Final action recommendations ---")
    if moran.p_sim < 0.05:
        print("**Conclusion: p-value less than 0.05, OLS model residuals show significant spatial autocorrelation (spatial clustering).**")
        print("   This means your global model failed to capture all spatial patterns, and the 'internal form-external status' relationship varies across different regions in the UK.")
        print("   **Strongly recommend proceeding to next step: Use MGWR to explore and explain this interesting spatial heterogeneity.**")
    else:
        print("**Conclusion: p-value greater than or equal to 0.05, OLS model residuals are randomly distributed in space.**")
        print("   This means your global OLS model has well explained the variable relationships, which are relatively stable spatially.")
        print("   **You do not need to proceed with MGWR analysis, current OLS results are sufficient as final, robust conclusions.**")

# --- Run main function ---
run_final_spatial_diagnostics()

In [ ]:
# moran_i_test_for_gamma_model.py

import pandas as pd
import geopandas as gpd
import statsmodels.formula.api as smf
import numpy as np
from libpysal.weights import Queen
from esda.moran import Moran

def run_spatial_diagnostics_for_gamma():
    """
    This script specifically performs spatial diagnostics for the Gamma model.
    Core logic:
    1. Load data and ensure geographic coordinate system is EPSG:2770.
    2. Run log-transformed OLS regression model using Gamma.
    3. Extract model residuals.
    4. Perform Moran's I spatial autocorrelation test on residuals.
    5. Report test results.
    """
    print("--- Spatial diagnostics for Gamma model regression residuals ---")

    # --- 1. File paths and model configuration ---
    ANALYSIS_DATA_FILE = 'master_analysis_data_final_cleaned.csv' 
    TTWA_BOUNDARY_FILE = "boundary/Travel_to_Work_Areas_Dec_2011_FCB_in_United_Kingdom_2022.geojson"
    
    # Core difference: model formula using Gamma
    MODEL_FORMULA_GAMMA = 'log_in_degree_weighted ~ Centripetality_Gamma_mean + jobs_density'

    # --- 2. Load, transform coordinate system and merge data ---
    print("\n>>> Loading data and setting correct coordinate system (EPSG:2770)...")
    try:
        df_analysis = pd.read_csv(ANALYSIS_DATA_FILE)
        gdf_ttwa = gpd.read_file(TTWA_BOUNDARY_FILE)
        
        gdf_ttwa_projected = gdf_ttwa.to_crs('EPSG:2770')
        gdf_analysis = gdf_ttwa_projected.merge(df_analysis, left_on='TTWA11NM', right_on='TTWA_Name', how='inner')
        
        print(f"   Data loading and merging successful, {len(gdf_analysis)} TTWAs for spatial analysis.")

    except FileNotFoundError as e:
        print(f"Error: File {e.filename} not found. Please check path.")
        return
    except Exception as e:
        print(f"Error during data loading or merging: {e}")
        return

    # --- 3. Run OLS regression and extract residuals ---
    print("\n>>> Running Gamma OLS model to obtain residuals...")
    
    gdf_analysis['log_in_degree_weighted'] = np.log1p(gdf_analysis['in_degree_weighted'])
    
    try:
        model = smf.ols(formula=MODEL_FORMULA_GAMMA, data=gdf_analysis).fit()
        gdf_analysis['residuals_gamma'] = model.resid
        print("   OLS model (Gamma) run completed, residuals successfully extracted.")
    except Exception as e:
        print(f"Error running OLS model: {e}")
        return

    # --- 4. Build spatial weight matrix (W) ---
    print("\n>>> Building spatial weight matrix (W)...")
    try:
        w = Queen.from_dataframe(gdf_analysis)
        w.transform = 'r'
        print("   Spatial weight matrix (W) built successfully.")
    except Exception as e:
        print(f"Error building spatial weight matrix: {e}")
        return

    # --- 5. Calculate Moran's I ---
    print("\n>>> Performing Moran's I test on Gamma model residuals...")
    try:
        moran_gamma = Moran(gdf_analysis['residuals_gamma'], w)
        print("   Moran's I calculation completed.")
    except Exception as e:
        print(f"Error calculating Moran's I: {e}")
        return

    # --- 6. Report results ---
    print("\n--- Spatial autocorrelation test results (Gamma model) ---")
    print(f"Moran's I statistic: {moran_gamma.I:.4f}")
    print(f"p-value (simulation): {moran_gamma.p_sim:.4f}")
    print("---------------------------------------")

    if moran_gamma.p_sim < 0.05:
        print("**Conclusion: p-value less than 0.05, Gamma model residuals also show significant spatial autocorrelation.**")
        print("   This provides further, independent evidence for your need to use MGWR and other spatial models.")
    else:
        print("**Conclusion: p-value greater than or equal to 0.05, Gamma model residuals are randomly distributed in space.**")


# --- Run main function ---
run_spatial_diagnostics_for_gamma()

##### MGWR

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from mgwr.sel_bw import Sel_BW
from mgwr.gwr import GWR, MGWR
import matplotlib.pyplot as plt
import warnings

In [ ]:
def create_mgwr_visualization(gdf, independent_vars, model_name, mgwr_results=None):
    """
    Create MGWR coefficient maps and local R² maps with improved color schemes
    """
    print(f"Step 3/3: Creating MGWR visualization results...")
    
    # Calculate local R² if mgwr_results is provided
    if mgwr_results is not None:
        print("Calculating local R² using alternative method...")
        
        # Method 1: Use residuals to calculate local R²
        try:
            y_obs = gdf['log_in_degree_weighted'].values
            y_pred = mgwr_results.mu.flatten() if hasattr(mgwr_results, 'mu') else None
            
            if y_pred is None:
                # Try to get predicted values from fitted values
                if hasattr(mgwr_results, 'fitted_values'):
                    y_pred = mgwr_results.fitted_values.flatten()
                elif hasattr(mgwr_results, 'predy'):
                    y_pred = mgwr_results.predy.flatten()
                else:
                    # Manual calculation using parameters
                    X = np.column_stack([np.ones(len(gdf)), 
                                        gdf[independent_vars[0]].values,
                                        gdf[independent_vars[1]].values])
                    y_pred = np.sum(mgwr_results.params * X, axis=1)
            
            # Calculate local R² using moving window approach
            from scipy.spatial.distance import cdist
            centroids = gdf.geometry.centroid
            coords = np.column_stack([centroids.x, centroids.y])
            
            # Calculate distance matrix
            distances = cdist(coords, coords)
            
            # Use adaptive bandwidth for local R² calculation
            bandwidths = distances.mean(axis=1) * 0.5  # Adaptive local window
            
            local_r2 = []
            for i in range(len(y_obs)):
                # Find neighbors within bandwidth
                neighbors = distances[i] <= bandwidths[i]
                if neighbors.sum() > 3:  # Need at least 4 points
                    y_local = y_obs[neighbors]
                    y_pred_local = y_pred[neighbors]
                    
                    # Calculate local R²
                    ss_res = np.sum((y_local - y_pred_local) ** 2)
                    ss_tot = np.sum((y_local - y_local.mean()) ** 2)
                    r2_local = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
                    local_r2.append(max(0, min(1, r2_local)))  # Bound between 0 and 1
                else:
                    local_r2.append(0.5)  # Default value for isolated points
            
            gdf[f'mgwr_local_R2_{model_name}'] = local_r2
            print(f"   Local R² calculated successfully (range: {np.min(local_r2):.3f} - {np.max(local_r2):.3f})")
            
        except Exception as e:
            print(f"   Failed to calculate local R²: {e}")
            # Fallback: use global R² as approximation
            global_r2 = mgwr_results.R2 if hasattr(mgwr_results, 'R2') else 0.5
            gdf[f'mgwr_local_R2_{model_name}'] = global_r2
            print(f"   Using global R² ({global_r2:.3f}) as approximation")
    
    # Define custom elegant color schemes
    def get_diverging_colormap(data_values):
        """Get appropriate elegant diverging colormap based on data range"""
        vmin, vmax = np.min(data_values), np.max(data_values)
        
        if vmin >= 0:
            # All positive values: use elegant warm colors (light peach to coral)
            return 'Oranges', vmin, vmax
        elif vmax <= 0:
            # All negative values: use elegant cool colors (light blue to navy)
            return 'Blues', vmin, vmax
        else:
            # Mixed values: use elegant diverging colormap (soft blue-white-pink)
            abs_max = max(abs(vmin), abs(vmax))
            return 'RdYlBu_r', -abs_max, abs_max
    
    # Create figure with coefficients and local R²
    n_vars = len(independent_vars)
    fig, axes = plt.subplots(2, n_vars + 1, figsize=(6*(n_vars + 1), 12))
    
    if n_vars == 1:
        axes = axes.reshape(-1, 2)
    
    # Plot coefficient maps with improved color schemes
    for i, var in enumerate(independent_vars):
        coeff_col = f'mgwr_coeff_{var}_{model_name}'
        if coeff_col in gdf.columns:
            coeff_values = gdf[coeff_col].values
            
            # Get appropriate colormap and normalization
            cmap, vmin, vmax = get_diverging_colormap(coeff_values)
            
            # Coefficient map with elegant colors
            im = gdf.plot(column=coeff_col, ax=axes[0, i], legend=True,
                         cmap=cmap, vmin=vmin, vmax=vmax,
                         legend_kwds={'shrink': 0.6, 'label': f'{var} Coefficient', 'aspect': 15},
                         edgecolor='lightgray', linewidth=0.2, alpha=0.8)
            
            axes[0, i].set_title(f'MGWR: {var} Coefficient', fontweight='bold', fontsize=14)
            axes[0, i].set_axis_off()
            
            # Enhanced statistics text with better formatting
            mean_coeff = np.mean(coeff_values)
            std_coeff = np.std(coeff_values)
            median_coeff = np.median(coeff_values)
            
            # Color-code the statistics based on mean value (softer colors)
            text_color = '#8B4513' if mean_coeff > 0 else '#4682B4'  # Saddle brown / Steel blue
            bg_color = '#FFF8DC' if mean_coeff > 0 else '#F0F8FF'    # Cornsilk / Alice blue
            
            stats_text = f'Mean: {mean_coeff:.4f}\nMedian: {median_coeff:.4f}\nStd: {std_coeff:.4f}\nRange: [{np.min(coeff_values):.4f}, {np.max(coeff_values):.4f}]'
            
            axes[1, i].text(0.5, 0.5, stats_text,
                           transform=axes[1, i].transAxes, ha='center', va='center', 
                           fontsize=11, color=text_color,
                           bbox=dict(boxstyle="round,pad=0.4", facecolor=bg_color, alpha=0.8))
            axes[1, i].set_title(f'{var} Statistics', fontweight='bold', fontsize=14, color=text_color)
            axes[1, i].set_axis_off()
    
    # Plot local R² map with warm color scheme
    r2_col = f'mgwr_local_R2_{model_name}'
    if r2_col in gdf.columns:
        r2_values = gdf[r2_col].values
        
        # Use elegant warm colors for R² (cream to soft orange)
        gdf.plot(column=r2_col, ax=axes[0, -1], legend=True,
                 cmap='YlOrRd', legend_kwds={'shrink': 0.6, 'label': 'Local R²', 'aspect': 15},
                 edgecolor='lightgray', linewidth=0.2, alpha=0.8)
        axes[0, -1].set_title(f'MGWR: Local R²', fontweight='bold', fontsize=14, color='#CD853F')
        axes[0, -1].set_axis_off()
        
        # R² statistics with enhanced formatting
        mean_r2 = np.mean(r2_values)
        global_r2 = mgwr_results.R2 if mgwr_results else "N/A"
        median_r2 = np.median(r2_values)
        
        r2_text = f'Global R²: {global_r2:.4f}\nMean Local R²: {mean_r2:.4f}\nMedian Local R²: {median_r2:.4f}\nLocal R² Range: [{np.min(r2_values):.4f}, {np.max(r2_values):.4f}]'
        
        axes[1, -1].text(0.5, 0.5, r2_text,
                        transform=axes[1, -1].transAxes, ha='center', va='center', 
                        fontsize=11, color='#CD853F',
                        bbox=dict(boxstyle="round,pad=0.4", facecolor="#FFFACD", alpha=0.8))
        axes[1, -1].set_title('Model Performance', fontweight='bold', fontsize=14, color='#CD853F')
        axes[1, -1].set_axis_off()
    
    plt.tight_layout()
    plt.savefig(f'mgwr_analysis_with_R2_{model_name}_elegant.png', dpi=300, bbox_inches='tight')
    print(f"Elegant MGWR analysis with R² map saved: mgwr_analysis_with_R2_{model_name}_elegant.png")
    plt.close()
    
    # Create separate enhanced local R² map
    if r2_col in gdf.columns:
        fig, ax = plt.subplots(1, 1, figsize=(14, 10))
        
        # Enhanced R² map with soft gradient background
        gdf.plot(column=r2_col, ax=ax, legend=True, cmap='Oranges', 
                 legend_kwds={'shrink': 0.5, 'label': 'Local R²', 'orientation': 'horizontal', 
                             'pad': 0.05, 'aspect': 20},
                 edgecolor='lightgray', linewidth=0.3, alpha=0.85)
        
        ax.set_title(f'MGWR Local R² Map - {model_name} Model', 
                    fontweight='bold', fontsize=18, color='#CD853F', pad=20)
        ax.set_axis_off()
        
        # Enhanced text box with gradient background
        r2_stats = f'Global R²: {global_r2:.4f}\nMean Local R²: {mean_r2:.4f}\nMedian Local R²: {median_r2:.4f}\nStd Local R²: {np.std(r2_values):.4f}\nMin-Max: {np.min(r2_values):.3f} - {np.max(r2_values):.3f}'
        
        # Create a more sophisticated text box with elegant colors
        props = dict(boxstyle='round,pad=0.5', facecolor='#FFFACD', alpha=0.9, 
                    edgecolor='#DEB887', linewidth=2)
        ax.text(0.02, 0.98, r2_stats, transform=ax.transAxes, fontsize=13,
                verticalalignment='top', bbox=props, color='#8B7355', fontweight='bold')
        
        plt.tight_layout()
        plt.savefig(f'map_local_R2_{model_name}_elegant.png', dpi=300, bbox_inches='tight')
        print(f"Elegant local R² map saved: map_local_R2_{model_name}_elegant.png")
        plt.close()
    
    # Create individual coefficient maps with enhanced styling
    for i, var in enumerate(independent_vars):
        coeff_col = f'mgwr_coeff_{var}_{model_name}'
        if coeff_col in gdf.columns:
            fig, ax = plt.subplots(1, 1, figsize=(14, 10))
            
            coeff_values = gdf[coeff_col].values
            cmap, vmin, vmax = get_diverging_colormap(coeff_values)
            
            # Enhanced individual coefficient map with elegant styling
            gdf.plot(column=coeff_col, ax=ax, legend=True,
                     cmap=cmap, vmin=vmin, vmax=vmax,
                     legend_kwds={'shrink': 0.5, 'label': f'{var} Coefficient', 
                                 'orientation': 'horizontal', 'pad': 0.05, 'aspect': 20},
                     edgecolor='lightgray', linewidth=0.3, alpha=0.85)
            
            # Determine title color based on coefficient mean (elegant colors)
            title_color = '#B8860B' if np.mean(coeff_values) > 0 else '#5F9EA0'  # Dark goldenrod / Cadet blue
            
            ax.set_title(f'MGWR Coefficient Map: {var} - {model_name} Model', 
                        fontweight='bold', fontsize=18, color=title_color, pad=20)
            ax.set_axis_off()
            
            # Enhanced statistics box with elegant colors
            mean_coeff = np.mean(coeff_values)
            stats_text = f'Mean: {mean_coeff:.4f}\nMedian: {np.median(coeff_values):.4f}\nStd: {np.std(coeff_values):.4f}\nRange: [{np.min(coeff_values):.4f}, {np.max(coeff_values):.4f}]'
            
            bg_color = '#FFFACD' if mean_coeff > 0 else '#F0F8FF'  # Lemon chiffon / Alice blue
            text_color = '#8B7355' if mean_coeff > 0 else '#4682B4'  # Dark khaki / Steel blue
            edge_color = '#DEB887' if mean_coeff > 0 else '#87CEEB'  # Burlywood / Sky blue
            
            props = dict(boxstyle='round,pad=0.5', facecolor=bg_color, alpha=0.9, 
                        edgecolor=edge_color, linewidth=2)
            ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, fontsize=13,
                    verticalalignment='top', bbox=props, color=text_color, fontweight='bold')
            
            plt.tight_layout()
            clean_var_name = var.replace('_', ' ').replace('mean', 'Mean')
            plt.savefig(f'map_coeff_{clean_var_name}_{model_name}_elegant.png', dpi=300, bbox_inches='tight')
            print(f"Elegant coefficient map saved: map_coeff_{clean_var_name}_{model_name}_elegant.png")
            plt.close()

def run_compatible_mgwr(gdf, dependent_var, independent_vars, model_name):
    """
    More compatible MGWR implementation with proper results return
    """
    print(f"\n{'='*20} Compatibility MGWR Analysis ({model_name}) {'='*20}")
    
    # --- 1. Data Preparation ---
    y = gdf[dependent_var].values.reshape(-1, 1)
    X = gdf[independent_vars].values
    
    # Add intercept term
    X_with_intercept = np.column_stack([np.ones(X.shape[0]), X])
    
    # Calculate centroid coordinates
    centroids = gdf.geometry.centroid
    coords = np.column_stack([centroids.x, centroids.y])
    
    print(f"Data shapes - Y: {y.shape}, X: {X_with_intercept.shape}, Coordinates: {coords.shape}")
    
    # --- 2. Smart Bandwidth Strategy ---
    print(">>> Using smart multiscale bandwidth strategy...")
    
    try:
        # Strategy 1: Try native multiscale selection directly
        print("   Strategy 1: Trying native MGWR multiscale...")
        bw_selector = Sel_BW(coords, y, X_with_intercept, multi=True)
        multi_bws = bw_selector.search(verbose=False)
        
        # Check return value type
        if hasattr(multi_bws, '__len__') and len(multi_bws) == X_with_intercept.shape[1]:
            bws = list(multi_bws)
            print(f"   Successfully obtained multiscale bandwidth: {bws}")
            mgwr_method = "native_multi"
        else:
            raise ValueError("Mismatch in bandwidth count")
            
    except Exception as e1:
        print(f"   Strategy 1 failed: {e1}")
        
        try:
            # Strategy 2: Stepwise obtain bandwidth for different variables
            print("   Strategy 2: Stepwise multiscale bandwidth selection...")
            bws = []
            
            # Overall model bandwidth as baseline
            base_selector = Sel_BW(coords, y, X_with_intercept)
            base_bw = base_selector.search()
            
            # Use larger bandwidth for intercept (global trend)
            bws.append(base_bw * 2.0)
            
            # Calculate bandwidth for each independent variable
            for i, var in enumerate(independent_vars):
                # Create matrix with intercept and current variable
                X_subset = np.column_stack([np.ones(X.shape[0]), X[:, i]])
                var_selector = Sel_BW(coords, y, X_subset)
                var_bw = var_selector.search()
                
                # Adjust bandwidth to reflect variable characteristics
                if i == 0:  # First variable (usually the main explanatory variable)
                    adjusted_bw = var_bw * 0.7  # More local influence
                else:  # Second variable
                    adjusted_bw = var_bw * 1.1  # Slightly more global influence
                    
                bws.append(adjusted_bw)
                print(f"   {var} bandwidth: {adjusted_bw:.2f}")
                
            print(f"   Stepwise multiscale bandwidth: {bws}")
            mgwr_method = "stepwise_multi"
            
        except Exception as e2:
            print(f"   Strategy 2 also failed: {e2}")
            
            # Strategy 3: Fixed strategy based on data density
            print("   Strategy 3: Adaptive fixed bandwidth strategy...")
            n_points = coords.shape[0]
            
            # Calculate average nearest neighbor distance
            from scipy.spatial.distance import pdist
            distances = pdist(coords)
            avg_distance = np.mean(distances)
            
            # Adapt based on data density
            density_factor = np.sqrt(n_points / 100)  # Normalize to 100 points
            
            bws = [
                avg_distance * density_factor * 1.5,  # Intercept - global
                avg_distance * density_factor * 0.6,  # First variable - local
                avg_distance * density_factor * 1.0   # Second variable - medium
            ]
            
            print(f"   Adaptive bandwidth strategy: {bws}")
            mgwr_method = "adaptive_fixed"
    
    # --- Fix: Ensure bws is list or np.ndarray ---
    if not isinstance(bws, (list, np.ndarray)) or len(bws) != X_with_intercept.shape[1]:
        bws = [bws] * X_with_intercept.shape[1]

    # --- 3. Run MGWR Model ---
    print(f">>> Fitting MGWR model using {mgwr_method} method...")
    
    try:
        # Ensure bandwidth is positive
        bws = [max(bw, 1.0) for bw in bws]
        
        # Use selector object instead of bandwidth list
        selector = Sel_BW(coords, y, X_with_intercept, multi=True)
        selector.search(verbose=True)  # Execute bandwidth search first, results will be saved internally
        mgwr_model = MGWR(coords, y, X_with_intercept, selector)
        mgwr_results = mgwr_model.fit()
        print("   MGWR model fitted successfully!")
        print(f"   Global R²: {mgwr_results.R2:.4f}")
        print(f"   Adjusted R²: {getattr(mgwr_results, 'adj_R2', 'N/A')}")
        print(f"   AIC: {mgwr_results.aic:.2f}")
        print(f"   Effective degrees of freedom: {mgwr_results.tr_S:.2f}")
        
        # Store coefficients
        param_names = ['Intercept'] + independent_vars
        for i, param in enumerate(param_names):
            coeff_col = f'mgwr_coeff_{param}_{model_name}'
            gdf[coeff_col] = mgwr_results.params[:, i]
            coeff_mean = mgwr_results.params[:, i].mean()
            coeff_std = mgwr_results.params[:, i].std()
            coeff_range = mgwr_results.params[:, i].max() - mgwr_results.params[:, i].min()
            print(f"   {param} coefficient statistics:")
            print(f"     Mean: {coeff_mean:.4f}, Std: {coeff_std:.4f}, Range: {coeff_range:.4f}")
        
        # Store bandwidth info
        if hasattr(selector, 'bw'):
            if isinstance(selector.bw, (list, np.ndarray)):
                bandwidths = {param: selector.bw[i] for i, param in enumerate(param_names)}
            else:
                bandwidths = {param: selector.bw for param in param_names}
            print(f"Variable bandwidths: {bandwidths}")
            # Store bandwidth information as string
            gdf[f'mgwr_bandwidth_{model_name}'] = str(bandwidths)
        else:
            print("No bandwidth information found")
            gdf[f'mgwr_bandwidth_{model_name}'] = "Not available"
        
        gdf[f'mgwr_method_{model_name}'] = mgwr_method
        return mgwr_results, mgwr_method
        
    except Exception as e:
        print(f"MGWR fitting failed: {e}")
        print("Detailed error information:")
        import traceback
        traceback.print_exc()
        return None, None

def compare_gwr_mgwr(gdf, dependent_var, independent_vars, model_name):
    """
    Compare GWR and MGWR results
    """
    print(f"\n{'='*20} GWR vs MGWR Comparison Analysis ({model_name}) {'='*20}")
    
    y = gdf[dependent_var].values.reshape(-1, 1)
    X = gdf[independent_vars].values
    centroids = gdf.geometry.centroid
    coords = np.column_stack([centroids.x, centroids.y])
    
    results = {}
    
    # 1. Standard GWR
    print(">>> Running standard GWR...")
    try:
        bw_gwr = Sel_BW(coords, y, X).search()
        gwr_model = GWR(coords, y, X, bw_gwr).fit()
        
        results['GWR'] = {
            'model': gwr_model,
            'R2': gwr_model.R2,
            'AIC': gwr_model.aic,
            'bandwidth': bw_gwr
        }
        
        print(f"   GWR - R²: {gwr_model.R2:.4f}, AIC: {gwr_model.aic:.2f}, Bandwidth: {bw_gwr:.2f}")
        
    except Exception as e:
        print(f"   GWR failed: {e}")
    
    # 2. MGWR
    print(">>> Running MGWR...")
    mgwr_results, mgwr_method = run_compatible_mgwr(gdf, dependent_var, independent_vars, model_name)
    
    if mgwr_results:
        results['MGWR'] = {
            'model': mgwr_results,
            'R2': mgwr_results.R2,
            'AIC': mgwr_results.aic,
            'method': mgwr_method
        }
        
        # Generate visualization with the results
        create_mgwr_visualization(gdf, independent_vars, model_name, mgwr_results)
    
    # 3. Compare results
    if len(results) > 1:
        print(f"\n>>> Model Comparison Results:")
        print(f"{'Model':<10} {'R²':<10} {'AIC':<10} {'Note'}")
        print("-" * 50)
        
        for model_name_comp, result in results.items():
            note = "Better fit" if result['R2'] == max(r['R2'] for r in results.values()) else ""
            print(f"{model_name_comp:<10} {result['R2']:<10.4f} {result['AIC']:<10.2f} {note}")
    
    return results

def main():
    # --- Environment Check ---
    print("=== MGWR Analysis Environment Check ===")
    mgwr_available = check_mgwr_version()
    
    if not mgwr_available:
        print("MGWR compatibility issues detected, will use compatibility mode")
    else:
        print("MGWR environment check passed")
    
    # --- File Path Configuration ---
    ANALYSIS_DATA_FILE = 'master_analysis_data_final_cleaned.csv' 
    TTWA_BOUNDARY_FILE = "boundary/Travel_to_Work_Areas_Dec_2011_FCB_in_United_Kingdom_2022.geojson"

    # --- Load Data ---
    print("\n=== Data Loading and Preprocessing ===")
    try:
        df_analysis = pd.read_csv(ANALYSIS_DATA_FILE)
        gdf_ttwa = gpd.read_file(TTWA_BOUNDARY_FILE)
        
        # Use British National Grid coordinate system
        gdf_ttwa_projected = gdf_ttwa.to_crs('EPSG:27700')
        gdf_master = gdf_ttwa_projected.merge(df_analysis, left_on='TTWA11NM', right_on='TTWA_Name', how='inner')
        
        # Log transformation
        gdf_master['log_in_degree_weighted'] = np.log1p(gdf_master['in_degree_weighted'])
        
        print(f"Data preparation completed, {len(gdf_master)} TTWAs in total")
        print(f"Dependent variable statistics: mean={gdf_master['log_in_degree_weighted'].mean():.3f}, std={gdf_master['log_in_degree_weighted'].std():.3f}")
        
        # Check data completeness
        for var in ['Anisotropy_Lambda_mean', 'Centripetality_Gamma_mean', 'jobs_density']:
            if var in gdf_master.columns:
                missing = gdf_master[var].isna().sum()
                if missing > 0:
                    print(f"Warning: {var} has {missing} missing values")
                    gdf_master[var] = gdf_master[var].fillna(gdf_master[var].median())
                    print(f"Filled missing values in {var} with median")
        
    except Exception as e:
        print(f"Data loading failed: {e}")
        return

    # --- MGWR Analysis ---
    print(f"\n{'='*60}")
    print("Starting MGWR multiscale geographically weighted regression analysis")
    print(f"{'='*60}")
    
    # Lambda model MGWR analysis
    print("\nLambda Model MGWR Analysis")
    gdf_lambda = gdf_master.copy()
    compare_gwr_mgwr(
        gdf=gdf_lambda,
        dependent_var='log_in_degree_weighted',
        independent_vars=['Anisotropy_Lambda_mean', 'jobs_density'],
        model_name='Lambda'
    )
    
    # Save Lambda model results
    lambda_output = 'mgwr_lambda_results.geojson'
    mgwr_cols = [col for col in gdf_lambda.columns if 'mgwr_' in col or col in ['geometry', 'TTWA11NM', 'TTWA_Name']]
    gdf_lambda[mgwr_cols].to_file(lambda_output, driver='GeoJSON')
    print(f"Lambda MGWR results saved: {lambda_output}")
    
    # Gamma model MGWR analysis
    print("\nGamma Model MGWR Analysis")
    gdf_gamma = gdf_master.copy()
    compare_gwr_mgwr(
        gdf=gdf_gamma,
        dependent_var='log_in_degree_weighted',
        independent_vars=['Centripetality_Gamma_mean', 'jobs_density'],
        model_name='Gamma'
    )
    
    # Save Gamma model results
    gamma_output = 'mgwr_gamma_results.geojson'
    mgwr_cols = [col for col in gdf_gamma.columns if 'mgwr_' in col or col in ['geometry', 'TTWA11NM', 'TTWA_Name']]
    gdf_gamma[mgwr_cols].to_file(gamma_output, driver='GeoJSON')
    print(f"Gamma MGWR results saved: {gamma_output}")
    
    print(f"\n{'='*60}")
    print("MGWR analysis completed!")
    print("Generated elegant color scheme map files:")
    print("Comprehensive analysis charts:")
    print("   - mgwr_analysis_with_R2_Lambda_elegant.png")
    print("   - mgwr_analysis_with_R2_Gamma_elegant.png")
    print("Elegant local R² maps:")
    print("   - map_local_R2_Lambda_elegant.png")
    print("   - map_local_R2_Gamma_elegant.png")
    print("Elegant coefficient maps:")
    print("   - map_coeff_Anisotropy Lambda Mean_Lambda_elegant.png")
    print("   - map_coeff_jobs density_Lambda_elegant.png")
    print("   - map_coeff_Centripetality Gamma Mean_Gamma_elegant.png")
    print("   - map_coeff_jobs density_Gamma_elegant.png")
    print("Complete result files:")
    print("   - mgwr_lambda_results.geojson")
    print("   - mgwr_gamma_results.geojson")
    print(f"{'='*60}")

if __name__ == '__main__': 
    main()